In [1]:
import util as u

In [2]:
df = u.read_file("adagrad")
na_df = df[df["result_err"].isna()]
val_total_math_iter = int(na_df[na_df["count"] == 10]["math_iters"].mean() / 1000)
df.insert(
    loc= len(df.columns),
    column='math_per_oper',
    value=[val_total_math_iter for i in range(df.shape[0])])
df.head()

,count,alpha,run,name,res_point,first_err,result_err,delta_time,math_iters,math_per_oper
0,10,1,1001,adagrad,"[1.610085490747614, 1.0834146570765388]",22300.993847,NaN,0.105904,12000,12
1,10,2,1001,adagrad,"[1.9723492111005987, 1.005835118172833]",22300.993847,NaN,0.099676,12000,12
2,10,50,154,adagrad,"[1.992674664495263, 1.0015749708301231]",22300.993847,0.000093,0.017888,1836,12
3,10,100,185,adagrad,"[1.9924971094431048, 1.0015124045938186]",22300.993847,0.000097,0.014375,2208,12
4,10,1000000,135,adagrad,"[2.0074059678864877, 0.998352874268054]",22300.993847,0.000097,0.015693,1608,12


In [19]:
def toFixed(numObj, digits=0):
    return f"{numObj:.{digits}f}"


# df.head(5)
print(df.head(5).to_latex(
    formatters={"res_point": lambda x: [float(toFixed(i, 1)) for i in x]} 
                         ))

\begin{tabular}{lrrrllrrrr}
\toprule
 & count & alpha & run & name & res_point & first_err & result_err & delta_time & math_iters \\
\midrule
0 & 10 & 1 & 1001 & adagrad & [1.6, 1.1] & 22300.993847 & NaN & 0.105904 & 12000 \\
1 & 10 & 2 & 1001 & adagrad & [2.0, 1.0] & 22300.993847 & NaN & 0.099676 & 12000 \\
2 & 10 & 50 & 154 & adagrad & [2.0, 1.0] & 22300.993847 & 0.000093 & 0.017888 & 1836 \\
3 & 10 & 100 & 185 & adagrad & [2.0, 1.0] & 22300.993847 & 0.000097 & 0.014375 & 2208 \\
4 & 10 & 1000000 & 135 & adagrad & [2.0, 1.0] & 22300.993847 & 0.000097 & 0.015693 & 1608 \\
\bottomrule
\end{tabular}



решено было взять такие $\alpha$ так в силу того что *adagrad* использует в себе корретирование $\alpha$ и при меньшем алгоритм не успевает сойтись

In [6]:
#get_alpha(df)
df["alpha"].unique()

array([      1,       2,      50,     100, 1000000], dtype=int64)

для испытания кода, бралось 4 точки, (±10, ±10), ниже представлен код когда не сошлось

In [8]:
df.groupby('alpha')['result_err'].apply(lambda x: x.isna().sum())

alpha
1          28
2          25
50          0
100         0
1000000     0
Name: result_err, dtype: int64

In [11]:
print(df.groupby('alpha')['result_err'].apply(lambda x: x.isna().sum())\
.to_latex())

\begin{tabular}{lr}
\toprule
 & result_err \\
alpha &  \\
\midrule
1 & 28 \\
2 & 25 \\
50 & 0 \\
100 & 0 \\
1000000 & 0 \\
\bottomrule
\end{tabular}



далее удалим те методы которые не сошлись из статистики

In [52]:
df_na = df.dropna()

среднее значение итераций, для каждого количества точек и выбрынных $\alpha$

In [53]:
df_na.groupby(['count', 'alpha'])['run'].apply(lambda x: x.mean())

count  alpha  
10     50         206.25
       100        213.50
       1000000    245.00
40     50         224.50
       100        247.50
       1000000    234.25
70     50         179.25
       100        193.00
       1000000    201.00
100    2          874.00
       50         165.25
       100        158.75
       1000000    170.25
130    2          834.00
       50         151.50
       100        170.75
       1000000    182.00
160    50         197.50
       100        207.75
       1000000    230.25
190    2          851.00
       50         156.75
       100        164.25
       1000000    170.75
Name: run, dtype: float64

In [54]:
df_na.groupby(['count', 'alpha'])['delta_time'].apply(lambda x: x.mean())

count  alpha  
10     50         0.020884
       100        0.021578
       1000000    0.026606
40     50         0.051529
       100        0.055989
       1000000    0.053997
70     50         0.068468
       100        0.072524
       1000000    0.075452
100    2          0.504441
       50         0.099972
       100        0.084320
       1000000    0.095548
130    2          0.717907
       50         0.110240
       100        0.124477
       1000000    0.134439
160    50         0.197378
       100        0.206002
       1000000    0.228659
190    2          1.066304
       50         0.194170
       100        0.207005
       1000000    0.219662
Name: delta_time, dtype: float64

In [56]:
df_na.groupby(['count', 'alpha'])['delta_time'].apply(lambda x: x.mean()).mean()

0.19323133180538812

In [55]:
df_na.groupby(['count', 'alpha'])['math_iters'].apply(lambda x: x.mean())

count  alpha  
10     50          2463.0
       100         2550.0
       1000000     2928.0
40     50          2682.0
       100         2958.0
       1000000     2799.0
70     50          2139.0
       100         2304.0
       1000000     2400.0
100    2          10476.0
       50          1971.0
       100         1893.0
       1000000     2031.0
130    2           9996.0
       50          1806.0
       100         2037.0
       1000000     2172.0
160    50          2358.0
       100         2481.0
       1000000     2751.0
190    2          10200.0
       50          1869.0
       100         1959.0
       1000000     2037.0
Name: math_iters, dtype: float64

In [60]:
df_na.groupby(['count', 'alpha'])['math_iters'].apply(lambda x: x.mean()).mean()

3302.5

как стало видно, почти все методы имеют около 3к математических действий и примерно 0.2 мс на выполнение